In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [9]:
import boto3

bucket_name = 'onpassive777rr'  # Replace with your desired unique bucket name

# Get the region of the instance
my_region = boto3.session.Session().region_name
print(my_region)

s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-2':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
        print('S3 bucket created successfully')
except Exception as e:
    if 'BucketAlreadyExists' in str(e):
        print(f'S3 error: Bucket with name {bucket_name} already exists. Choose a different name.')
    else:
        print('S3 error: ', e)


us-east-2
S3 bucket created successfully


In [10]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://onpassive777rr/xgboost-as-a-built-in-algo/output


In [11]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [12]:

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
import os
import pandas as pd
import boto3
import sagemaker

# Assuming you already have 'train_data' DataFrame defined

# Save train data to a local CSV file
train_data[['y_yes'] + [col for col in train_data.columns if col not in ['y_no', 'y_yes']]].to_csv('train.csv', index=False, header=False)

# Set your S3 bucket name and prefix
bucket_name =  'onpassive777rr'    # Replace with your S3 bucket name
prefix = 'your-prefix'  # Replace with your desired prefix

# Upload the local CSV file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Define the S3 input for SageMaker training using TrainingInput
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [15]:
import os
import pandas as pd
import boto3
import sagemaker

# Assuming you already have 'test_data' DataFrame defined

# Save test data to a local CSV file
test_data[['y_yes'] + [col for col in test_data.columns if col not in ['y_no', 'y_yes']]].to_csv('test.csv', index=False, header=False)

# Set your S3 bucket name and prefix
bucket_name = 'onpassive777rr'  # Replace with your S3 bucket name
prefix = 'your-prefix'  # Replace with your desired prefix

# Upload the local CSV file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Define the S3 input for SageMaker testing using TrainingInput
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')


### Building Models Xgboot- Inbuilt Algorithm

In [16]:
import sagemaker

container = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=boto3.Session().region_name,
    version="1.0-1"
)

In [17]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [19]:
estimator = sagemaker.estimator.Estimator(image_uri=container,  # Use image_uri instead of image_name
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5,  # Use volume_size instead of train_volume_size
                                          output_path=output_path,
                                          use_spot_instances=True,  # Use use_spot_instances instead of train_use_spot_instances
                                          max_run=300,  # Use max_run instead of train_max_run
                                          max_wait=600)  # Use max_wait instead of train_max_wait


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [20]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-11-22-06-08-15-938


2023-11-22 06:08:16 Starting - Starting the training job...
2023-11-22 06:08:31 Starting - Preparing the instances for training.........
2023-11-22 06:09:47 Downloading - Downloading input data...
2023-11-22 06:10:28 Training - Training image download completed. Training in progress...[2023-11-22 06:10:49.394 ip-10-0-127-95.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node

In [21]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-11-22-06-18-22-059
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-11-22-06-18-22-059
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-11-22-06-18-22-059


------!

In [29]:
print(xgb_predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker-xgboost-2023-11-22-06-18-22-059


In [30]:
endpoint_name = 'sagemaker-xgboost-2023-11-22-06-18-22-059'
print(f"Endpoint URL: {xgb_predictor.endpoint}")


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Endpoint URL: sagemaker-xgboost-2023-11-22-06-18-22-059


In [25]:
from sagemaker.serializers import CSVSerializer

In [26]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values
xgb_predictor.serializer = CSVSerializer()

# Assuming xgb_predictor is your SageMaker Predictor instance
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)

(12357,)


In [27]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [28]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [31]:
bucket_name

'onpassive777rr'

In [32]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-11-22-06-18-22-059


[{'ResponseMetadata': {'RequestId': 'T5YEJD7F1N25JM3E',
   'HostId': 'aQm0Tk0n7VKqNbhAoSQtSTJOGieiU7TsQxA3/gwx1aExcU9gPgYvf4xSsqD8WNmFzZxhHxOCZ/A=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'aQm0Tk0n7VKqNbhAoSQtSTJOGieiU7TsQxA3/gwx1aExcU9gPgYvf4xSsqD8WNmFzZxhHxOCZ/A=',
    'x-amz-request-id': 'T5YEJD7F1N25JM3E',
    'date': 'Wed, 22 Nov 2023 06:35:52 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-11-22-06-08-15-938/debug-output/collections/000000000/worker_0_collections.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-11-22-06-08-15-938/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-11-22-06-08-15-938/debug-output/i